In [2]:
#webscrape the link into pandas df
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize

import numpy as np

import json
from bs4 import BeautifulSoup
import requests
print('Imported')

Imported


In [3]:
#Use BeautifulSoup to transform the data
link = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#make beautifulsoup object
soup = BeautifulSoup(link, 'html.parser')

#create lists to store the data
postCodeList =[]
boroughList = []
neighborhoodList = []

In [4]:
#add to the lists
for row in soup.find('table').find_all('tr'):
    cell = row.find_all('td')
    if(len(cell) > 0):
        postCodeList.append(cell[0].text)
        boroughList.append(cell[1].text)
        neighborhoodList.append(cell[2].text.rstrip('\n'))

In [5]:
#Read the dataframe
canada_df = pd.DataFrame({'Postal Code': postCodeList,
                          'Borough': boroughList,
                          'Neighborhood': neighborhoodList})
canada_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [7]:
#Drop cells with 'Not assigned'
canada_df_drop = canada_df[canada_df.Borough != 'Not assigned'].reset_index(drop=True)
canada_df_drop.head()


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [8]:
#using lambda to aggregate the neighborhoods; first time incorporating lambda
canada_df_group = canada_df_drop.groupby(['Postal Code', 'Borough'], as_index=False).agg(lambda x: ", ".join(x))
   
canada_df_group.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
#assign not assigned neighborhood to respective borough. 
#iterate over the rows 

for ind, row in canada_df_group.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] == row['Borough']
        
canada_df_group.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
coord = pd.read_csv("https://cocl.us/Geospatial_data")
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
canada_df_group2 = canada_df_group.merge(coord, on="Postal Code", how="left")
canada_df_group2.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [12]:
import matplotlib as mpl
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
print("Done")

Done


In [13]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print("Done")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.0               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

In [14]:
place = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(place)
lat = location.latitude
long = location.longitude
print("The coordinates of Toronto are {}, {}".format(lat, long))

The coordinates of Toronto are 43.653963, -79.387207


In [15]:
map_tor = folium.Map(location = [lat, long], zoom_start=10)
for neighborhood, borough in canada_df_group2.items():
    label = '{}, {}'.format(neighborhood,borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color="#fc4503",
        fill_opacity=0.7).add_to(map_tor)
                                          
map_tor

In [16]:
bor_name = list(canada_df_group2.Borough.unique())

tor_bor = []

for boro in bor_name:
    if 'toronto' in boro.lower():
        tor_bor.append(boro)
        
canada_df_group2 = canada_df_group2[canada_df_group2['Borough'].isin(tor_bor)].reset_index(drop=True)
canada_df_group2.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [17]:
map_tor = folium.Map(location = [lat, long], zoom_start=10)
for lat, long, neighborhood, borough in zip(canada_df_group2['Latitude'], canada_df_group2['Longitude'], canada_df_group2['Borough'], canada_df_group2['Neighborhood']):
    label = '{}, {}'.format(neighborhood,borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color="#fc4503",
        fill_opacity=0.7).add_to(map_tor)
map_tor
